## docker image
Perform action on Docker images. To start with, we can pull an image with `docker image pull [OPTIONS] NAME[:TAG|@DIGEST]`. For example: 
```bash
$ docker image pull alpine
Using default tag: latest
latest: Pulling from library/alpine
96526aa774ef: Pull complete
Digest: sha256:eece025e432126ce23f223450a0326fbebde39cdf496a85d8c016293fc851978
Status: Downloaded newer image for alpine:latest
docker.io/library/alpine:latest
```

To view all images, use `docker image ls [OPTIONS] [REPOSITORY[:TAG]]`:
```bash
$ docker image ls
REPOSITORY   TAG       IMAGE ID       CREATED       SIZE
alpine       latest    8ca4688f4f35   6 weeks ago   7.34MB
```

Where did the image get stored? We can query this using `docker info`:
```bash
$docker info
...
Storage Driver: overlay2
Docker Root Dir: /var/lib/docker
...
```

To remove an image, we use `docker image rm [OPTIONS] IMAGE [IMAGE...]`
```bash
$ docker image rm alpine:latest
Untagged: alpine:latest
Untagged: alpine@sha256:eece025e432126ce23f223450a0326fbebde39cdf496a85d8c016293fc851978
Deleted: sha256:8ca4688f4f356596b5ae539337c9941abc78eda10021d35cbc52659c74d9b443
Deleted: sha256:cc2447e1835a40530975ab80bb1f872fbab0f2a0faecf2ab16fbbb89b3589438
```

## docker run
Provides all the launch capabilities for Docker. Syntax: `docker run [OPTIONS] IMAGE [COMMAND] [ARG...]`. The options list is quite big, however some commonly used ones are:
<div style="display: inline-block">

| Option        | Short | Description                                          |
|---------------|-------|------------------------------------------------------|
| --detach      | -d    | Runs container in background and prints container ID |
| --env         | -e    | Set environment variables                            |
| --interactive | -i    | Keep STDIN open even if not attached                 |
| --label       | -l    | Set container metadata                               |
| --name        |       | Assign name to container                             |
| --rm          |       | Remove container after completion                    |
| --tty         | -t    | Allocate a pseudo TTY                                |
| --volume      | -v    | Bind mount a volume                                  |

```bash
$ docker run -i -t alpine /bin/sh
Unable to find image 'alpine:latest' locally
latest: Pulling from library/alpine
96526aa774ef: Pull complete
Digest: sha256:eece025e432126ce23f223450a0326fbebde39cdf496a85d8c016293fc851978
Status: Downloaded newer image for alpine:latest
/ # hostname
6d233d75ec08
/ #
```

In the above example, Alpine image was not available on machine, so Docker downloaded it. Immediately after the container was ready sh was made ready providing an interctive shell in the new container. This container has a network, IP address and a bridge interface to talk to the local host.

The above container runs as long as the command specified `/bin/sh` is running. If we type `exit`, the container would stop running.

To create a long running daemonized container, use the `-d` option:
```bash
$ docker run --name looping_daemon -d alpine /bin/sh -c "while true; do echo Hello World; sleep 1; done"
f31e5733e4979d136fa7e5acb9db310f530b1d222c7de78f439ece2bf872cd68
$ docker container ls
CONTAINER ID   IMAGE     COMMAND                  CREATED         STATUS         PORTS     NAMES
f31e5733e497   alpine    "/bin/sh -c 'while t…"   9 seconds ago   Up 7 seconds             looping_daemon
```

While the container is running, we can use `docker logs [OPTIONS] CONTAINER` to view logs generated by the container. Use the `--tail 10` option to view last 10 log lines.

To view the container's running processes, we can use `docker top CONTAINER [ps OPTIONS]` command:
```bash
$ docker top looping_daemon
UID        PID        PPID        C        STIME        TTY        TIME        CMD
root       7020       6999        0        20:27        ?          00:00:00    /bin/sh -c while true; do echo Hello World; sleep 1; done
root       7046       7020        0        20:27        ?          00:00:00    sleep 1
```

Sometimes we may need to run some command inside an already running container - to do this use `docker exec [OPTIONS] CONTAINER COMMAND [ARG...]`. It supports some similar options (compared to docker run):
<div style="display: inline-block">

| Option        | Short | Description                                  |
|---------------|-------|----------------------------------------------|
| --detach      | -d    | Detached mode: run command in the background |
| --env         | -e    | Set environment variables                    |
| --interactive | -i    | Keep STDIN open even if not attached         |
| --tty         | -t    | Allocate a pseudo TTY                        |
| --workdir     | -w    | Working directory inside the container       |

We can use `docker exec` command to run maintenance, monitoring or management tasks inside running container.

**Bind mounts** lets us bind a directory on host machine in the container. The file or directory does not need to exist on the Docker host already. It is created on demand if it does not yet exist. The `-v` or `--volume` command is used to specify bind volume. It has three parts separated by comma:
- absolute path to directory/file on host machine
- path where directory/file is mounted on container
- optional comma separated list of options such as `ro`, `z`, `Z`.

If we bind-mount a directory into a non-empty directory on the container, the directory's existing contents are obscured by the bind mount. This can cause issues if we use some well defined directory as mount point:
```bash
$ docker run -d -it --name broken-container -v /tmp:/usr nginx:latest
docker: Error response from daemon: oci runtime error: container_linux.go:262: starting container process caused "exec: \"nginx\": executable file not found in $PATH".
```

Example usage of bind mount:
```bash
$ pwd
/home/testuser/docker
$ ls /home/testuser/docker
Dockerfile
$ docker run -it -v /home/salmananjum/docker:/docker alpine sh
/ # ls /docker/
Dockerfile
/ # exit
```

## docker container
Lists containers created by us. Syntax: `docker container ls [OPTIONS]`. By default, this command would only show running containers, to view all containers, use `--all` option.

```bash
$docker container ls --all
CONTAINER ID   IMAGE     COMMAND     CREATED         STATUS                     PORTS     NAMES
6d233d75ec08   alpine    "/bin/sh"   9 minutes ago   Exited (0) 3 minutes ago             sad_bohr
```

Notice that Docker provides a unique friendly name to the created container. In fact there are three ways to refer to a container:
- short UUID 6d233d75ec08
- longer UUID
- or name like sad_bohr

To start a previously stopped container, use `docker container start [OPTIONS] CONTAINER [CONTAINER...]`. Our container will start with the same options we'd specified when we launched using `docker run`. So, in the previous case, we have an interactive sesssion waiting on our running container, we need to reattach to that session using `--attach` option.
```bash
$ docker container start --attach sad_bohr
/ #
```

There are two ways to stop a runnig container, use `docker container stop [OPTIONS] CONTAINER [CONTAINER...]` or use `docker container kill [OPTIONS] CONTAINER [CONTAINER...]`. The former is more graceful, it gives the running process chance to perform cleanup before exiting. The latter is more abrupt in nature.

To remove a container, use the `docker container rm [OPTIONS] CONTAINER [CONTAINER...]` command. It returns name of the removed container after removing the container. Use the `--force` option to kill the container if it is running.

To get more information about the container, we can use `docker container inspect` command - it spits out a large JSON response containing a lot of information. Which is why it is preferable to use `--format` option to output only the desired attributes:
```bash
$ docker container inspect --format='{{ .State.Running }}' sad_bohr
false
$ docker container inspect --format='{{ .NetworkSettings.IPAddress }}' looping_daemon
172.17.0.2
```

## docker volume
Earlier we discussed about *bind mounts* which provide a way to share files between the host machine and docker. Volumes are the preferred mechanism for persisting data generated by and used by Docker containers becuase the alternative *bind mounts* are dependent on the directory structure and OS of the host machine.

To create a Docker volume, use the `docker volume create [OPTIONS] [VOLUME]` command. Newly created volume is available at `/var/lib/docker/volumes`:

```bash
$ docker volume create sample_volume
sample_volume
$ ls -ltra /var/lib/docker/volumes
total 40
drwx-----x  3 root root  4096 Nov 15 23:47 9b48ce47eab794574eafbb581dd6f00c32c1002d35f5b54697ca3c32e5b9d8d7
drwx--x--- 13 root root  4096 Nov 17 21:42 ..
brw-------  1 root root 8, 32 Nov 17 21:42 backingFsBlockDev
drwx-----x  4 root root  4096 Nov 17 22:24 .
drwx-----x  3 root root  4096 Nov 17 22:24 sample_volume
-rw-------  1 root root 32768 Nov 17 22:24 metadata.db
```

To view all volumes user `docker volume ls [OPTIONS]` command, and to remove existing volume, use `docker volume rm [OPTIONS] VOLUME [VOLUME...]`.

We can use a previously created volume and use its name as first parameter of `-v` command. This will mount the volume in the container at path specified by the second parameter. If we start a container with a volume that doesn't yet exist, Docker creates the volume for us:

```bash
$ docker run -v test_volume:/volume -it alpine sh
/ # ls /
bin     etc     lib     mnt     proc    run     srv     tmp     var
dev     home    media   opt     root    sbin    sys     usr     volume
/ # echo "Random content" > /volume/random.txt
/ # exit
$ docker volume ls
DRIVER    VOLUME NAME
local     9b48ce47eab794574eafbb581dd6f00c32c1002d35f5b54697ca3c32e5b9d8d7
local     test_volume
$ sudo ls /var/lib/docker/volumes/test_volume/_data
random.txt
```